<a href="https://colab.research.google.com/github/zroe1/letterPredictor/blob/main/letterpredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow.compat.v2.feature_column as fc

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import clear_output
from six.moves import urllib

!pip install -U scikit-learn

In [23]:
training_df = pd.read_csv("https://raw.githubusercontent.com/zroe1/letterPredictor/main/training.csv")
eval_df = pd.read_csv("https://raw.githubusercontent.com/zroe1/letterPredictor/main/eval.csv")

letter5_from_training = training_df.pop("letter5")
letter5_from_eval = eval_df.pop("letter5")

# prints the head for the training dataframe (eval dataframe is same format)
print(training_df.head())
# NOTE: PoS stands for part of speech. The number in the part of speech
# column represents a letter which represents a part of speech
# (e.g, a = adjective)

   letter1  letter2  letter3  letter4  PoS
0       97       98      111      117  105
1      116      104      101      105   97
2      119      111      117      108  118
3      116      104      105      110  118
4      116      104      101      114  101


In [24]:
COLUMN_NAMES = ["letter1", "letter2", "letter3", "letter4", "PoS"] # All columns are numeric
feature_columns = []

for column_name in COLUMN_NAMES:
  feature_columns.append(tf.feature_column.numeric_column(column_name, dtype=tf.float32))

print(feature_columns)

[NumericColumn(key='letter1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='letter2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='letter3', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='letter4', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PoS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [25]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000) #randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds # return a batch of the dataset
  return input_function # return a function object for use

train_input_function = make_input_fn(training_df, letter5_from_training)
eval_input_function = make_input_fn(eval_df, letter5_from_eval, num_epochs=1, shuffle=False)

In [26]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns, n_classes=256)

In [32]:
linear_est.train(train_input_function)
result = linear_est.evaluate(eval_input_function)

clear_output()
print(result['accuracy'])

0.12096774


In [50]:
result = list(linear_est.predict(eval_input_function))
print(result[0]["probabilities"][120])

0.000399769
